In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import cv2
import numpy as np
import os
from PIL import Image
import psutil

In [ ]:

batch_size = 16
epochs = 50
learning_rate = 0.01
num_classes = 2

In [ ]:

train_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\Marcos_Squezee\Teste'
val_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\Marcos_Squezee\Treino'


In [ ]:
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.225], std=[0.229, 0.224, 0.225]),
    ]),
}


In [ ]:

train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:

classes = train_dataset.classes
print(f"Número de classes: {len(classes)}")
print(f"Classes: {classes}")

In [ ]:

model = models.squeezenet1_1(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, num_classes, kernel_size=(1, 1)),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d((1, 1))
)
model.num_classes = num_classes


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [ ]:

def visualize_predictions(model, loader, classes):
    model.eval()
    inputs, labels = next(iter(loader))
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

    for i in range(min(4, inputs.size(0))):
        img = inputs[i].cpu().numpy().transpose((1, 2, 0))
        img = (img * 0.229 + 0.485).clip(0, 1)
        plt.imshow(img)
        plt.title(f'Pred: {classes[preds[i]]} | True: {classes[labels[i]]}')
        plt.show()

In [ ]:

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

    model.eval()
    correct = 0
    total = 0
    class_correct = [0] * num_classes
    class_total = [0] * num_classes

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")
    for i in range(num_classes):
        print(f"Accuracy of {classes[i]}: {100 * class_correct[i] / class_total[i]:.2f}%")


In [ ]:

torch.save(model.state_dict(), "squeezenet_model.pth")
print("Modelo treinado e salvo!")

In [ ]:

MODEL_PATH = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\projeto\squeezenet_model.pth'
LABELS = ["Nao Reconhecido", "Meu Rosto"]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.squeezenet1_1(pretrained=False)
model.classifier[1] = torch.nn.Conv2d(512, len(LABELS), kernel_size=(1, 1), stride=(1, 1))
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def predict(frame):
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    input_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_tensor)
    _, predicted_class = torch.max(output, 1)
    return LABELS[predicted_class.item()]

In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Erro ao acessar a câmera!")
    exit()

print("Pressione 'q' para sair.")

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
epochs = 10
num_classes = 2
train_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\Marcos_Squezee\Teste'
val_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\Marcos_Squezee\Treino'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = models.squeezenet1_1(pretrained=False)
model.classifier[1] = torch.nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):

        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = running_loss / len(train_loader)
        avg_train_accuracy = 100 * correct_train / total_train
        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {avg_train_accuracy:.2f}%")

        model.eval()
        running_loss_val = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss_val += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        avg_val_loss = running_loss_val / len(val_loader)
        avg_val_accuracy = 100 * correct_val / total_val
        print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.2f}%")

train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs)


In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar o quadro!")
        break

    start_time = time.time()

    label = predict(frame)

    end_time = time.time()
    latency = end_time - start_time
    cpu_usage = psutil.cpu_percent()

    cv2.putText(frame, f"Predicao: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Latencia: {latency:.4f}s", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Uso CPU: {cpu_usage}%", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Camera - Teste do Modelo", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()